## Using SIMD operations and "rotate" gate to perform vector dot product

A powerful feature of some HE schemes is the ability to perform SIMD operations, doing the same calculation on multiple "slots" (i.e. elements of a vector).  The first part of a vector dot product - the component-wise multiplication - is therefore trivial.  However, we then need to sum over the elements to obtain the scalar product.  This can be done using ROTATE and ADD operations, as demonstrated in this notebook.

In [27]:
'''Step2. Load libraries for the python code'''

import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  print("Please set environment variable SHEEP_HOME to point to location of SHEEP/frontend")
import sys
sys.path.append(SHEEP_HOME)

from pysheep import sheep_client

In [28]:
'''
Step3. Start a new job

Yuanyuan Sun comment:
The step ID here is based on my operation/experiment. Whenever you need to start a new job, 
you need to perform this step. 
'''
sheep_client.new_job()

{'status_code': 200, 'content': ''}

In [30]:
'''
Step 4. Use the HE(homomorphic encryption) scheme which is SEAL_BFV
For the second experiment: the step ID is Step A. 
'''
sheep_client.set_context("SEAL_BFV")
sheep_client.set_input_type("int8_t")
sheep_client.get_nslots()

{'status_code': 200, 'content': {'nslots': 2048}}

With this set of parameters, we have 7 slots available, so can do a dot product of two 7-component vectors.
The circuit to perform this operation will be a MULTIPLY followed by a sequence of 6 ROTATEs and ADDs.

In [31]:
circuit = """
INPUTS input_0 input_1
CONST_INPUTS rotate_1
OUTPUTS output

input_0 input_1 MULTIPLY prod_r0

prod_r0 rotate_1 ROTATE prod_r1
prod_r0 prod_r1 ADD prod_s1

prod_r1 rotate_1 ROTATE prod_r2
prod_s1 prod_r2 ADD prod_s2

prod_r2 rotate_1 ROTATE prod_r3
prod_s2 prod_r3 ADD prod_s3

prod_r3 rotate_1 ROTATE prod_r4
prod_s3 prod_r4 ADD prod_s4

prod_r4 rotate_1 ROTATE prod_r5
prod_s4 prod_r5 ADD prod_s5

prod_r5 rotate_1 ROTATE prod_r6
prod_s5 prod_r6 ADD output
"""

In [32]:
sheep_client.set_circuit_text(circuit)
sheep_client.get_inputs()

{'status_code': 200, 'content': ['input_0', 'input_1']}

So the two input vectors are called input_0 and input_1. Let's assign them the values {2,4,6,8,10,1,3} and {1,0,0,0,0,0,0}

In [99]:
'''Step B. So the two input vectors are called input_0 and input_1. Let's assign them the values {2,4,6,8,10,1,3} and {1,0,0,0,0,0,0}'''

sheep_client.set_inputs({"input_0": [2,4,6,8, 10, 1, 3], "input_1": [1,0,0,0, 0, 0, 0]})

{'status_code': 200, 'content': ''}

In [100]:
sheep_client.get_const_inputs()

{'status_code': 200, 'content': ['rotate_1']}

The circuit also takes a "const input" (that won't be encrypted) - this is how much we will ROTATE the vector by in each step, so just set it to -1

In [101]:
sheep_client.set_const_inputs({"rotate_1": -1})

{'status_code': 200, 'content': ''}

In [102]:
'''Step C. Run the job'''
sheep_client.run_job()

{'status_code': 200, 'content': ''}

In [103]:
'''Step D. Get the result the client wants retrieve'''

sheep_client.get_results()

{'status_code': 200,
 'content': {'cleartext check': {'is_correct': True},
  'outputs': {'output': ['2,2,2,2,2,2,2'],
   'prod_s1': ['2,0,0,0,0,0,2'],
   'prod_s2': ['2,0,0,0,0,2,2'],
   'prod_s3': ['2,0,0,0,2,2,2']},
  'timings': {'decryption': '2693.107000',
   'encryption': '3170.730000',
   'evaluation': '16901.922000',
   'output': '11.385000',
   'prod_r0': '8029.834000',
   'prod_r1': '1076.429000',
   'prod_r2': '1062.248000',
   'prod_r3': '1088.404000',
   'prod_r4': '1089.470000',
   'prod_r5': '1123.074000',
   'prod_r6': '1145.714000',
   'prod_s1': '148.204000',
   'prod_s2': '365.494000',
   'prod_s3': '240.137000',
   'prod_s4': '316.884000',
   'prod_s5': '270.625000',
   'prod_s6': '302.586000'}}}

In [38]:
''' '''
sheep_client.set_inputs({"input_0": [2,4,6,8, 10, 1, 3], "input_1": [1,0,0,0, 0, 0, 0]})

sheep_client.get_const_inputs()

sheep_client.set_const_inputs({"rotate_1": -1})

sheep_client.run_job()

sheep_client.get_results()

{'status_code': 200,
 'content': {'cleartext check': {'is_correct': True},
  'outputs': {'output': ['2,2,2,2,2,2,2']},
  'timings': {'decryption': '1486.130000',
   'encryption': '3492.800000',
   'evaluation': '21384.373000',
   'output': '546.166000',
   'prod_r0': '11067.923000',
   'prod_r1': '1203.153000',
   'prod_r2': '1186.493000',
   'prod_r3': '1301.018000',
   'prod_r4': '1223.178000',
   'prod_r5': '1181.029000',
   'prod_r6': '1498.328000',
   'prod_s1': '221.043000',
   'prod_s2': '199.332000',
   'prod_s3': '399.859000',
   'prod_s4': '356.411000',
   'prod_s5': '368.924000'}}}

So, the output is ['2,2,2,2,2,2,2'] (we always get an output vector the same length as our input vector, even though in this case we only need one number), and 2 is indeed the scalar product of [2,4,6,8,10,1,3]  and [1,0,0,0,0,0,0].  


## Generalizing, and generating circuits

We probably don't want to write circuits by hand, particularly if we are dealing with long vectors (HElib and SEAL can, depending on parameter choices, offer thousands of slots).   We can write a simple python function to generate circuits for arbitrary length vectors:

In [73]:
'''Step 5. Generalizing, and generating circuits'''

def generate_vector_dot_product_circuit(input_0, input_1):
    """
    Given two input lists (must be equal in length) generate a SHEEP circuit to do the dot product
    """
    if len(input_0) != len(input_1):
        raise RuntimeError("input_0 and input_1 must be the same length")
    circuit_str = "INPUTS input_0 input_1\nCONST_INPUTS rotate_1\nOUTPUTS output prod_s1 prod_s2 prod_s3\ninput_0 input_1 MULTIPLY prod_r0\n"
    for i in range(len(input_0)-1):
        circuit_str += "prod_r{} rotate_1 ROTATE prod_r{}\n".format(i,i+1)
        if i==0:
            circuit_str += "prod_r0 prod_r1 ADD prod_s1\n"
        else:
            circuit_str += "prod_s{} prod_r{} ADD prod_s{}\n".format(i,i+1,i+1)
    circuit_str += "prod_s{} ALIAS output\n".format(i+1)
    return circuit_str

(Note that this function is also available in ```pysheep/mid_level_benchmarks.py```)

So, let's do the calculation in SEAL_BFV - multiply 2 vectors with 7 elements each, where each element 

In [78]:
'''Step 6. Calculate Dot Product'''
'''Step 6.1 Input the server vector and the client vector'''

import random
input_0 = [2,4,6,8,10,1,3]
input_1 = [1, 0, 0, 0, 0, 0, 0]

Lets quickly do the calculation in the clear so we know what answer to expect:

In [79]:
sum = 0
for i in range(len(input_0)):
    sum += input_0[i]*input_1[i]
sum

2

In [86]:
'''Step 6.2 Start the new job for this retrieval and Use the HE library which is SEAL_BFV'''

sheep_client.new_job()

{'status_code': 200, 'content': ''}

In [87]:
''' Use the HE library which is SEAL_BFV'''

sheep_client.set_context("SEAL_BFV")

{'status_code': 200, 'content': ''}

In [88]:
sheep_client.set_input_type("int16_t")

{'status_code': 200, 'content': ''}

In [89]:
sheep_client.get_nslots()

{'status_code': 200, 'content': {'nslots': 2048}}

In [91]:
'''Step 6.3 Generate the doc product for this retrieval with the circuit'''

circuit = generate_vector_dot_product_circuit(input_0,input_1)
sheep_client.set_circuit_text(circuit)

{'status_code': 200, 'content': ''}

In [92]:
sheep_client.set_inputs({"input_0":input_0, "input_1": input_1})

{'status_code': 200, 'content': ''}

In [93]:
sheep_client.set_const_inputs({"rotate_1": -1})

{'status_code': 200, 'content': ''}

In [94]:
sheep_client.set_timeout(60)

{'status_code': 200, 'content': ''}

In [95]:
sheep_client.run_job()

{'status_code': 200, 'content': ''}

In [96]:
sheep_client.get_results()['content']["cleartext check"]["is_correct"]

True

In [97]:
sheep_client.get_results()['content']["outputs"]["output"][0].split(",")[0]

'2'

So we get the right answer!

In [98]:
sheep_client.get_results()["content"]["timings"]["evaluation"]

'16901.922000'